In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from datetime import datetime

### 加载数据

In [ ]:
data_dir = "../data/"
operation_train = pd.read_csv(data_dir+"operation_train_new.csv")
transaction_train = pd.read_csv(data_dir+"transaction_train_new.csv")
tag_train = pd.read_csv(data_dir+"tag_train_new.csv")
operation_round1 = pd.read_csv(data_dir+"operation_round1_new.csv")
transaction_round1 = pd.read_csv(data_dir+"transaction_round1_new.csv")

In [ ]:
operation_train.shape

In [ ]:
len(operation_train["UID"].unique())

In [ ]:
transaction_train.shape

In [ ]:
len(transaction_train["UID"].unique())

In [ ]:
tag_train.shape

In [ ]:
operation_round1.shape

In [ ]:
transaction_round1.shape

In [ ]:
operation_train.head(2)

In [ ]:
operation_train.info()

In [ ]:
operation_train.describe()

In [ ]:
len(operation_train["UID"].unique())

In [ ]:
transaction_train.info()

In [ ]:
transaction_train.sample(2)

In [ ]:
len(transaction_train["UID"].unique())

In [ ]:
tag_train.head()

### 数据合并

In [ ]:
operation_tag_train = pd.merge(operation_train, tag_train, how="left", on="UID")
operation_tag_train.to_csv(data_dir + "operation_tag_train.csv", index=False)

In [ ]:
print("number of operation tag:{}".format(len(operation_tag_train)))
operation_tag_train.sample(2)

In [ ]:
transaction_tag_train = pd.merge(transaction_train, tag_train, how="left", on="UID")
transaction_tag_train.to_csv(data_dir+"transaction_tag_train.csv", index=False)

In [ ]:
print("number of transaction tag:{}".format(len(transaction_tag_train)))
transaction_tag_train.sample(2)

In [ ]:
operation_transaction_tag = pd.merge(operation_tag_train, transaction_tag_train, how="left", on="UID")
operation_transaction_tag.to_csv(data_dir+"operation_transaction_tag_train.csv", index=False)

### 特征工程

In [ ]:
n_operation_train = operation_train.shape[0]
n_operaton_round1 = operation_round1.shape[0]
operation_data = pd.concat([operation_train, operation_round1]).reset_index(drop=True)
operation_data.shape

In [ ]:
operation_data_na = (operation_data.isnull().sum()/len(operation_data))*100
operation_data_na = operation_data_na.drop(operation_data_na[operation_data_na == 0].index).sort_values(ascending=True)
missing_operation_data = pd.DataFrame({"missing ratio": operation_data_na})
print("The number of missing item: {}".format(len(missing_operation_data)))
missing_operation_data

#### success

In [ ]:
operation_data["success"] = operation_data["success"].fillna(2)

#### device1 device2 device_code1 device_code2 device_code3

In [ ]:
uid_device = pd.DataFrame()
device = operation_data.groupby(["UID"]).count()["device2"]
uid_device["UID"] = device.index
uid_device["device"] = device.values

In [ ]:
operation_data.drop(["device1", "device2", "device_code1", "device_code2", "device_code3"], axis=1, inplace=True)
operation_data = pd.merge(operation_data, uid_device, how="left", on="UID")
operation_data["device"] = operation_data["device"].fillna(0)
mean = operation_data["device"].mean()
std = operation_data["device"].std()
operation_data["device"] = (operation_data["device"]-mean)/std

#### version

In [ ]:
operation_data.drop(["version"], axis=1, inplace=True)

#### ip1 ip1_sub1

In [ ]:
uid_ip1 = pd.DataFrame()
ip1 = operation_data.groupby(["UID"]).count()["ip1_sub"]
uid_ip1["UID"] = ip1.index
uid_ip1["ip1"] = ip1.values

In [ ]:
operation_data.drop(["ip1_sub", "ip1"], axis=1, inplace=True)
operation_data = pd.merge(operation_data, uid_ip1, how="left", on="UID")
operation_data["ip1"] = operation_data["ip1"].fillna(0)
mean = operation_data["ip1"].mean()
std = operation_data["ip1"].std()
operation_data["ip1"] = (operation_data["ip1"]-mean)/std


#### ip2 ip2_sub

In [ ]:
uid_ip2 = pd.DataFrame()
ip2 = operation_data.groupby(["UID"]).count()["ip2_sub"]
uid_ip2["UID"] = ip2.index
uid_ip2["ip2"] = ip2.values

In [ ]:
operation_data.drop(["ip2_sub", "ip2"], axis=1, inplace=True)
operation_data = pd.merge(operation_data, uid_ip2, how="left", on="UID")
operation_data["ip2"] = operation_data["ip2"].fillna(0)
mean = operation_data["ip2"].mean()
std = operation_data["ip2"].std()
operation_data["ip2"] = (operation_data["ip2"]-mean)/std

#### geo_code

In [ ]:
uid_geo = pd.DataFrame()
geo = operation_data.groupby(["UID"]).count()["geo_code"]
uid_geo["UID"] = geo.index
uid_geo["geo_code"] = geo.values

In [ ]:
operation_data.drop("geo_code", axis=1, inplace=True)
operation_data = pd.merge(operation_data, uid_geo, how="left", on="UID")
operation_data["geo_code"] = operation_data["geo_code"].fillna(0)
mean = operation_data["geo_code"].mean()
std = operation_data["geo_code"].std()
operation_data["geo_code"] = (operation_data["geo_code"] - mean)/std

#### mac1 mac2 wifi

In [ ]:
operation_data.drop(["mac1", "mac2", "wifi"], axis=1, inplace=True)

In [ ]:
operation_data_na = (operation_data.isnull().sum()/len(operation_data)) * 100
operation_data_na = operation_data_na.drop(operation_data_na[operation_data_na == 0].index).sort_values(ascending = True)
missing_operation_data = pd.DataFrame({"missing ratio": operation_data_na})
print("The number of missing item: {}".format(len(missing_operation_data)))
missing_operation_data

In [ ]:
n_transaction_train = transaction_train.shape[0]
n_transaction_round1 = transaction_round1.shape[0]
transaction_data = pd.concat((transaction_train, transaction_round1)).reset_index(drop=True)
transaction_data.shape

In [ ]:
transaction_data_na = (transaction_data.isnull().sum()/len(transaction_data)) * 100
transaction_data_na = transaction_data_na.drop(transaction_data_na[transaction_data_na == 0].index).sort_values(ascending = True)
missing_transaction_data = pd.DataFrame({"missing ratio": transaction_data_na})
print("the number of missing item: {}".format(len(missing_transaction_data)))
missing_transaction_data

#### trans_type2

In [ ]:
transaction_data["trans_type2"].unique()

In [ ]:
transaction_data["trans_type2"] = transaction_data["trans_type2"].fillna(100)

#### ip1_sub ip1

In [ ]:
len(transaction_data["ip1_sub"].unique())

In [ ]:
len(transaction_data["ip1"].unique())

In [ ]:
uid_ip = pd.DataFrame()
ip = transaction_data.groupby(["UID"]).count()["ip1_sub"]
uid_ip["UID"] = ip.index
uid_ip["ip"] = ip.values

In [ ]:
transaction_data.drop(["ip1_sub", "ip1"], axis=1, inplace=True)
transaction_data = pd.merge(transaction_data, uid_ip, how="left", on="UID")
transaction_data["ip"] = transaction_data["ip"].fillna(0)
mean = transaction_data["ip"].mean()
std = transaction_data["ip"].std()
transaction_data["ip"] = (transaction_data["ip"]-mean)/std

#### device1 device2 device_code1 device_code2 device_code3

In [ ]:
uid_device = pd.DataFrame()
device = transaction_data.groupby(["UID"]).count()["device2"]
uid_device["UID"] = device.index
uid_device["device"] = device.values

In [ ]:
transaction_data.drop(["device1", "device2", "device_code1", "device_code2", "device_code3"], axis=1, inplace=True)
transaction_data = pd.merge(transaction_data, uid_device, how="left", on="UID")
transaction_data["device"] = transaction_data["device"].fillna(0)
mean = transaction_data["device"].mean()
std = transaction_data["device"].std()
transaction_data["device"] = (transaction_data["device"]-mean)/std

#### geo_code

In [ ]:
uid_geo = pd.DataFrame()
geo = transaction_data.groupby(["UID"]).count()["geo_code"]
uid_geo["UID"] = geo.index
uid_geo["geo_code"] = geo.values

In [ ]:
transaction_data.drop(["geo_code"], axis=1, inplace=True)
transaction_data = pd.merge(transaction_data, uid_geo, how="left", on="UID")
transaction_data["geo_code"] = transaction_data["geo_code"].fillna(0)
mean = transaction_data["geo_code"].mean()
std = transaction_data["geo_code"].std()
transaction_data["geo_code"] = (transaction_data["geo_code"]-mean)/std

#### amt_src2

In [ ]:
uid_amt_src = pd.DataFrame()
amt_src2 = transaction_data.groupby("UID").count()["amt_src2"]
uid_amt_src["UID"] = amt_src2.index
uid_amt_src["amt_src2"] = amt_src2.values

In [ ]:
transaction_data.drop("amt_src2", axis=1, inplace=True)
transaction_data = pd.merge(transaction_data, uid_amt_src, how="left", on="UID")
transaction_data["amt_src2"] = transaction_data["amt_src2"].fillna(0)
mean = transaction_data["amt_src2"].mean()
std = transaction_data["amt_src2"].std()
transaction_data["amt_src2"] = (transaction_data["amt_src2"] - mean)/std

#### market_type

In [ ]:
transaction_data["market_type"] = transaction_data["market_type"].fillna(0)

In [ ]:
transaction_data.drop("market_code", axis=1, inplace=True)

#### else

In [ ]:
transaction_data.drop(["mac1", "acc_id1", "acc_id2", "acc_id3", "code1", "code2"], axis=1, inplace=True)

In [ ]:
transaction_data.drop("merchant", axis=1, inplace=True)

In [ ]:
transaction_data_na = (transaction_data.isnull().sum()/len(transaction_data)) * 100
transaction_data_na = transaction_data_na.drop(transaction_data_na[transaction_data_na == 0].index).sort_values(ascending = True)
missing_transaction_data = pd.DataFrame({"missing ratio": transaction_data_na})
print("The number of missing item: {}".format(len(missing_transaction_data)))
missing_transaction_data

### transform variables

In [ ]:
operation_data.sample(2)

In [ ]:
operation_data['time'] = pd.to_datetime(operation_data['time'])
operation_data['time'] = operation_data['time'].apply(lambda x:(x-datetime.now()).seconds/3600)

In [ ]:
mean = operation_data['time'].mean()
std = operation_data['time'].std()
operation_data['time'] = (operation_data['time'] - mean)/std

In [ ]:
operation_data['day'] = operation_data['day'].apply(str)
operation_data['mode'] = operation_data['mode'].apply(str)
operation_data['success'] = operation_data['success'].apply(str)
operation_data['os'] = operation_data['os'].apply(str)

In [ ]:
cols = ['day', 'mode', 'success', 'os']
for col in cols:
    le = LabelEncoder()
    le.fit(list(operation_data[col].values))
    operation_data[col]=le.transform(list(operation_data[col].values))

In [ ]:
transaction_data.sample(2)

In [ ]:
transaction_data['time'] = pd.to_datetime(transaction_data['time'])
transaction_data['time'] = transaction_data['time'].apply(lambda x:(x-datetime.now()).seconds/3600)

In [ ]:
cols = ['time', 'trans_amt', 'bal']
for col in cols:
    mean = transaction_data[col].mean()
    std = transaction_data[col].std()
    transaction_data[col] = (transaction_data[col] - mean)/std

In [ ]:
cols = ['channel', 'day', 'amt_src1', 'trans_type1', 'trans_type2', 'market_type']
for col in cols:
    le = LabelEncoder()
    le.fit(list(transaction_data[col].values))
    transaction_data[col] = le.transform(list(transaction_data[col].values))

In [ ]:
cols = ['day', 'mode', 'success', 'os']
for col in cols:
    operation_data[col] = operation_data[col].apply(str)

In [ ]:
operation_data = pd.get_dummies(operation_data)
operation_train = operation_data[:n_operation_train]
operation_round1 = operation_data[n_operation_train:]
operation_train.shape

In [ ]:
cols = ['channel', 'day', 'amt_src1', 'trans_type1', 'trans_type2', 'market_type']
for col in cols:
    transaction_data[col] = transaction_data[col].apply(str)

In [ ]:
transaction_data = pd.get_dummies(transaction_data)
transaction_train = transaction_data[:n_transaction_train]
transaction_round1 = transaction_data[n_transaction_train:]
transaction_train.shape

In [ ]:
operation_tag_train = pd.merge(operation_train, tag_train, how='left', left_on='UID',right_on='UID')
operation_tag_train.to_csv(data_dir + 'operation_tag_train.csv', index=False)

In [ ]:
transaction_tag_train = pd.merge(transaction_train, tag_train, how='left', left_on='UID',right_on='UID')
transaction_tag_train.to_csv(data_dir + 'transaction_tag_train.csv', index=False)

In [ ]:
operation_round1.to_csv(data_dir+"operation_round1.csv", index=False)
transaction_round1.to_csv(data_dir+"transaction_round1.csv", index=False)

### model deployment

In [ ]:
operation_round1.to_csv(data_dir+"operation_round1.csv", index=False)
transaction_round1.to_csv(data_dir+"transaction_round1.csv", index=False)

In [ ]:
import pandas as pd
operation_tag_train = pd.read_csv("../data/operation_tag_train.csv")
transaction_tag_train = pd.read_csv("../data/transaction_tag_train.csv")

In [ ]:
operation_round1 = pd.read_csv("../data/operation_round1.csv")
transaction_round1 = pd.read_csv("../data/transaction_round1.csv")

In [ ]:
operation_tag_train.head(2)

In [ ]:
operation_tag = operation_tag_train["Tag"]
operation_train = operation_tag_train.drop(["UID", "Tag"], axis=1)

In [ ]:
operation_train.shape

In [ ]:
transaction_tag = transaction_tag_train["Tag"]
transaction_train = transaction_tag_train.drop(["UID", "Tag"], axis=1)

In [ ]:
transaction_train.shape

In [ ]:
import catboost as cb
model1 = cb.CatBoostClassifier(iterations=2000,
                               learning_rate=0.05,
                               depth=3, 
                               l2_leaf_reg=4,
                               border_count=15,
                               loss_function="Logloss",
                               verbose=200)
cv_m1 = model1.fit(operation_train, operation_tag)

In [ ]:
model2 = cb.CatBoostClassifier(iterations=2000,
                               learning_rate=0.05,
                               depth=3,
                               l2_leaf_reg=4,
                               border_count=15,
                               loss_function="Logloss",
                               verbose=200)
cv_m2 = model2.fit(transaction_train, transaction_tag)

### predict

In [ ]:
submission = pd.read_csv("../data/sub.csv")
submission.head(2)

In [ ]:
uid = submission["UID"].to_frame()
uid_operation = pd.merge(uid, operation_round1, how="left", on=["UID"])
uid_transaction = pd.merge(uid, transaction_round1, how="left", on=["UID"])
operation_uid = uid_operation["UID"]
uid_operation.drop(["UID"], axis=1, inplace=True)
transaction_uid = uid_transaction["UID"]
uid_transaction.drop(["UID"], axis=1, inplace=True)

In [ ]:
operation_pred = model1.predict_proba(uid_operation)
transaction_pred = model2.predict_proba(uid_transaction)

In [ ]:
operation_sub = pd.DataFrame()
operation_sub["UID"] = operation_uid
operation_sub["Tag"] = operation_pred[:, 1]
operation_sub.head(2)

In [ ]:
transaction_sub = pd.DataFrame()
transaction_sub["UID"] = transaction_uid
transaction_sub["Tag"] = transaction_pred[:, 1]
transaction_sub.head(2)

In [ ]:
operation_sub = operation_sub.groupby("UID").mean()["Tag"]

In [ ]:
transaction_sub = transaction_sub.groupby("UID").mean()["Tag"]

In [ ]:
del operation_tag_train
del transaction_tag_train

del operation_round1
del transaction_round1

In [ ]:
ensemble = (operation_sub + transaction_sub)/2.0
ensemble

In [ ]:
submission = pd.DataFrame()
submission["UID"] = ensemble.index
submission["Tag"] = ensemble.values
submission.to_csv("../data/submission.csv", index=False)

## xbg

In [ ]:
import pandas as pd
operation_tag_train = pd.read_csv("../data/operation_tag_train.csv")
transaction_tag_train = pd.read_csv("../data/transaction_tag_train.csv")

operation_round1 = pd.read_csv("../data/operation_round1.csv")
transaction_round1 = pd.read_csv("../data/transaction_round1.csv")

operation_tag = operation_tag_train["Tag"]
operation_train = operation_tag_train.drop(["UID", "Tag"], axis=1)

transaction_tag = transaction_tag_train["Tag"]
transaction_train = transaction_tag_train.drop(["UID", "Tag"], axis=1)

In [ ]:
submission = pd.read_csv("../data/sub.csv")

uid = submission["UID"].to_frame()
uid_operation = pd.merge(uid, operation_round1, how="left", on=["UID"])
uid_transaction = pd.merge(uid, transaction_round1, how="left", on=["UID"])
operation_uid = uid_operation["UID"]
uid_operation.drop(["UID"], axis=1, inplace=True)

transaction_uid = uid_transaction["UID"]
uid_transaction.drop(["UID"], axis=1, inplace=True)



In [ ]:
import xgboost as xgb
params = {"booster": "gbtree",
          "objective" : "binary:logistic",
          "eval_metric" : "auc",
          "verbose" : 1,
          "eta" : 0.01,
          "max_delta_step" : 5,
          "max_depth" : 20,
          "alpha" : 10,
          "lambda" : 10,
          "gamma" : 2,
          "subsample" : 0.8,
          "colsample_bylevel" : 0.8,
          "min_child_weight" : 5,
          "scale_pos_weight" : 0.8,
          "n_jobs" : -1}


test_UID = operation_uid
test_X = uid_operation
train_matrix = xgb.DMatrix(operation_train, operation_tag)
test_matrix = xgb.DMatrix(test_X)

model1 = xgb.train(params, train_matrix, num_boost_round=3000)
operation_pred = model1.predict(test_matrix)

operation_sub = pd.DataFrame({"UID": test_UID, "Tag": operation_pred})

In [ ]:
params = {"booster": "gbtree",
          "objective" : "binary:logistic",
          "eval_metric" : "auc",
          "verbose" : 1,
          "eta" : 0.01,
          "max_delta_step" : 5,
          "max_depth" : 20,
          "alpha" : 10,
          "lambda" : 10,
          "gamma" : 2,
          "subsample" : 0.8,
          "colsample_bylevel" : 0.8,
          "min_child_weight" : 5,
          "scale_pos_weight" : 0.8,
          "n_jobs" : -1}


test_UID = transaction_uid
test_X = uid_transaction
train_matrix = xgb.DMatrix(transaction_train, transaction_tag)
test_matrix = xgb.DMatrix(test_X)

model2 = xgb.train(params, train_matrix, num_boost_round=3000)
transaction_pred = model2.predict(test_matrix)

transaction_sub = pd.DataFrame({"UID": test_UID, "Tag": transaction_pred})

In [ ]:
operation_sub = operation_sub.groupby("UID").mean()["Tag"]
transaction_sub = transaction_sub.groupby("UID").mean()["Tag"]

In [ ]:
ensemble = (operation_sub + transaction_sub)/2.0
submission = pd.DataFrame()
submission["UID"] = ensemble.index
submission["Tag"] = ensemble.values
submission.to_csv("../data/submission2.csv", index=False)

In [1]:
from sklearn import datasets
import pandas as pd
import xgboost as xgb
from sklearn.metrics import mean_squared_error

#使用sklearn自带的boston数据集
boston_data = datasets.load_boston()
df_boston = pd.DataFrame(boston_data.data,columns=boston_data.feature_names)
df_boston['target'] = pd.Series(boston_data.target)

xtrain = df_boston.head(500).drop(['target'],axis=1)
ytrain = df_boston.head(500).target

xtest = df_boston.tail(6).drop(['target'],axis=1)
ytest = df_boston.tail(6).target


In [2]:
params = {
    'eta':0.1,
    'max_depth':2,
    'min_child_weight':3,
    'gamma':0,
    'subsample':.8,
    'colsample_bytree':.7,
    'reg_alpha':1,
    'objective':'reg:linear'
}
dtrain = xgb.DMatrix(xtrain,ytrain)
dtest = xgb.DMatrix(xtest,ytest)
watchlist1 = [(dtrain,'train'),(dtest,'test')]

model1 = xgb.train(params=params,dtrain=dtrain,num_boost_round=100,early_stopping_rounds=10,evals=watchlist1)


[10:29:05] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pruned nodes, max_depth=2
[0]	train-rmse:21.7093	test-rmse:17.3474
Multiple eval metrics have been passed: 'test-rmse' will be used for early stopping.

Will train until test-rmse hasn't improved in 10 rounds.
[10:29:05] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pruned nodes, max_depth=2
[1]	train-rmse:19.6869	test-rmse:15.0228
[10:29:06] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pruned nodes, max_depth=2
[2]	train-rmse:17.8982	test-rmse:13.2654
[10:29:06] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pruned nodes, max_depth=2
[3]	train-rmse:16.2475	test-rmse:11.6048
[10:29:06] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning 

In [3]:
result1 = model1.predict(xgb.DMatrix(xtest),ntree_limit=model1.best_iteration)
mse1 = mean_squared_error(ytest,result1)

In [4]:
mse1

10.802571870358932

In [5]:
from xgboost.sklearn import XGBRegressor
model2 = XGBRegressor(
    learn_rate = 0.1,
    max_depth = 2,
    min_child_weight = 3,
    gamma = 0,
    subsample = 0.8,
    colsample_bytree = 0.7,
    reg_alpha = 1,
    objective = 'reg:linear',
    n_estimators = 100
)
watchlist2 = [(xtrain,ytrain),(xtest,ytest)]
model2.fit(xtrain,ytrain,eval_set=watchlist2,early_stopping_rounds=10)


[0]	validation_0-rmse:21.7093	validation_1-rmse:17.3474
Multiple eval metrics have been passed: 'validation_1-rmse' will be used for early stopping.

Will train until validation_1-rmse hasn't improved in 10 rounds.
[1]	validation_0-rmse:19.6869	validation_1-rmse:15.0228
[2]	validation_0-rmse:17.8982	validation_1-rmse:13.2654
[3]	validation_0-rmse:16.2475	validation_1-rmse:11.6048
[4]	validation_0-rmse:14.7695	validation_1-rmse:10.0835
[5]	validation_0-rmse:13.5417	validation_1-rmse:8.49356
[6]	validation_0-rmse:12.3675	validation_1-rmse:7.26111
[7]	validation_0-rmse:11.3112	validation_1-rmse:6.23273
[8]	validation_0-rmse:10.3767	validation_1-rmse:5.15305
[9]	validation_0-rmse:9.54068	validation_1-rmse:4.24451
[10]	validation_0-rmse:8.80674	validation_1-rmse:3.72274
[11]	validation_0-rmse:8.10549	validation_1-rmse:3.28673
[12]	validation_0-rmse:7.5088	validation_1-rmse:3.17986
[13]	validation_0-rmse:6.98417	validation_1-rmse:3.18837
[14]	validation_0-rmse:6.51555	validation_1-rmse:3.286

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.7, gamma=0, learn_rate=0.1, learning_rate=0.1,
       max_delta_step=0, max_depth=2, min_child_weight=3, missing=None,
       n_estimators=100, n_jobs=1, nthread=None, objective='reg:linear',
       random_state=0, reg_alpha=1, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=True, subsample=0.8)

In [6]:
result2 = model2.predict(xtest,ntree_limit=model2.best_iteration)
mse2 = mean_squared_error(ytest,result2)

In [7]:
mse2

10.802571870358932